In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Define paths
BASE_PATH = "C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/organized_dataset copy"
OUTPUT_PATH = "C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/mask2"

# Ensure output directories exist
os.makedirs(os.path.join(OUTPUT_PATH, "normal"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "pneumonia"), exist_ok=True)

def margin(img, margin_percent=10):
    """Crop 10% margins from all sides."""
    h, w = img.shape
    margin_x = int(w * margin_percent / 100)
    margin_y = int(h * margin_percent / 100)
    return img[margin_y:h-margin_y, margin_x:w-margin_x]

def process(img):
    """Apply KMeans clustering for segmentation."""
    flat_img = img.flatten().reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, n_init=10, random_state=0)
    labels = kmeans.fit_predict(flat_img)
    centers = kmeans.cluster_centers_
    
    if np.mean(centers[1]) > np.mean(centers[0]):
        labels = 1 - labels

    return np.reshape(labels, img.shape) * 255  # Convert to binary mask

def segment_and_filter(clustered_img):
    """Extract the two largest contours (lungs)."""
    clustered_img = clustered_img.astype(np.uint8)
    _, thresh = cv2.threshold(clustered_img, 0, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]  # Select 2 largest contours

    result = np.zeros_like(clustered_img)
    cv2.drawContours(result, contours, -1, 255, thickness=cv2.FILLED)
    return result, contours

def calculate_trapezium_area(top_width, bottom_width, height):
    """Calculate the area of the trapezium."""
    return 0.5 * (top_width + bottom_width) * height

def detect_and_plot_trapezium(processed_img):
    """Fit a trapezium around the largest lung contours and calculate area."""
    color_image = cv2.cvtColor(processed_img, cv2.COLOR_GRAY2BGR)
    _, binary_image = cv2.threshold(processed_img, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) < 2:
        return color_image  # Skip if fewer than 2 contours

    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]
    combined_rect = cv2.boundingRect(np.concatenate(contours))

    # Expand bounding rectangle for margin
    x, y, w, h = combined_rect
    combined_rect = (x - 20, y - 20, w + 40, h + 40)

    # Define trapezium points
    combined_trapezium_pts = np.array([
        (combined_rect[0], combined_rect[1] + combined_rect[3]),  # Bottom-left
        (combined_rect[0] + combined_rect[2], combined_rect[1] + combined_rect[3]),  # Bottom-right
        (combined_rect[0] + int(combined_rect[2] * 0.75), combined_rect[1]),  # Top-right
        (combined_rect[0] + int(combined_rect[2] * 0.25), combined_rect[1])  # Top-left
    ], np.int32)

    # Draw trapezium
    # cv2.polylines(color_image, [combined_trapezium_pts], isClosed=True, color=(0, 0, 255), thickness=3)

    # Calculate trapezium area
    top_width = combined_rect[2] * 0.5
    bottom_width = combined_rect[2]
    height = combined_rect[3]
    area = calculate_trapezium_area(top_width, bottom_width, height)

    print(f"Trapezium Area: {area:.2f}")

    return color_image

def process_image(image_path):
    """Process a single image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Skipping {image_path}, unable to load image.")
        return None

    cropped_img = margin(img)
    clustered_img = process(cropped_img)
    filtered_result, largest_contours = segment_and_filter(clustered_img)

    # Mirror left lung to right lung
    half_width = filtered_result.shape[1] // 2
    mirrored_half = cv2.flip(filtered_result[:, :half_width], 1)
    joined_image = cv2.hconcat([filtered_result[:, :half_width], mirrored_half])

    # Apply trapezium detection and visualization
    final_image = detect_and_plot_trapezium(joined_image)

    return final_image

def process_all_images():
    """Iterate through both folders and process all images."""
    for category in ["normal", "pneumonia"]:
        input_folder = os.path.join(BASE_PATH, category)
        output_folder = os.path.join(OUTPUT_PATH, category)

        for filename in os.listdir(input_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(input_folder, filename)
                processed_image = process_image(image_path)

                if processed_image is not None:
                    output_path = os.path.join(output_folder, filename)
                    cv2.imwrite(output_path, processed_image)
                    print(f"Saved: {output_path}")

# Run the processing function
process_all_images()


Trapezium Area: 958369.50
Saved: C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/mask2\normal\aug_0_9997.jpeg
Trapezium Area: 1046272.50
Saved: C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/mask2\normal\IM-0001-0001.jpeg
Trapezium Area: 1454520.00
Saved: C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/mask2\normal\IM-0003-0001.jpeg
Trapezium Area: 1684009.50
Saved: C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/mask2\normal\IM-0005-0001.jpeg
Trapezium Area: 1108672.50
Saved: C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/mask2\normal\IM-0006-0001.jpeg
Trapezium Area: 1864497.00
Saved: C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/mask2\normal\IM-0007-0001.jpeg
Trapezium Area: 1167390.00
Saved: C:/Users/tulsi/Desktop/Pneumonia-Detection-using-Deep-Learning - Copy/mask2\normal\IM-0009-0001.jpeg
Trapezium Area: 1252746.00
Saved: C:/Users/tulsi/Desktop/P

KeyboardInterrupt: 